# SageMaker JumpStart Foundation Models - Fine-tuning text generation GPT-J 6B model on domain specific dataset

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

---

---
Welcome to [Amazon SageMaker Built-in Algorithms](https://sagemaker.readthedocs.io/en/stable/algorithms/index.html)! You can use SageMaker Built-in algorithms to solve many Machine Learning tasks through [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html). You can also use these algorithms through one-click in SageMaker Studio via [JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html).

In this demo notebook, we demonstrate how to use the SageMaker Python SDK for finetuning Foundation Models and deploying the trained model for inference. The Foundation models perform Text Generation task. It takes a text string as input and predicts next words in the sequence.

* **How to run inference on [GPT-J 6B](https://huggingface.co/EleutherAI/gpt-j-6b) model without finetuning.**
* **How to fine-tune [GPT-J 6B](https://huggingface.co/EleutherAI/gpt-j-6b) model on a domain specific dataset, and then run inference on the fine-tuned model. In particular, the example dataset we demonstrated is [publicly available SEC filing](https://www.sec.gov/edgar/searchedgar/companysearch) of Amazon from year 2021 to 2022. The expectation is that after fine-tuning, the model should be able to generate insightful text in financial domain.**
* **We compare the inference result for GPT-J 6B before finetuning and after finetuning.**

Note: This notebook was tested on ml.t3.medium instance in Amazon SageMaker Studio with Python 3 (Data Science) kernel and in Amazon SageMaker Notebook instance with conda_python3 kernel.

---

1. [Set Up](#1.-Set-Up)
2. [Select Text Generation Model GTP-J 6B](#2.-Select-Text-Generation-Model-GTP-J-6B)
3. [Run Inference on the Pre-trained Model without finetuning](#3.-Run-Inference-on-the-Pre-trained-Model-without-finetuning)
    * [Retrieve Artifacts & Deploy an Endpoint](#3.1.-Retrieve-Artifacts-&-Deploy-an-Endpoint)
    * [Query endpoint and parse response](#3.2.-Query-endpoint-and-parse-response)
    * [Clean up the endpoint](#3.3.-Clean-up-the-endpoint)
4. [Finetune the pre-trained model on a custom dataset](#4.-Fine-tune-the-pre-trained-model-on-a-custom-dataset)
    * [Retrieve Training artifacts](#4.1.-Retrieve-Training-artifacts)
    * [Set Training parameters](#4.2.-Set-Training-parameters)
    * [Train with Automatic Model Tuning](#4.3.-Train-with-Automatic-Model-Tuning-([HPO]))
    * [Start Training](#4.4.-Start-Training)
    * [Extract Training performance metrics](#4.5.-Extract-Training-performance-metrics)
    * [Deploy & run Inference on the fine-tuned model](#4.6.-Deploy-&-run-Inference-on-the-fine-tuned-model)

## 1. Set Up
Before executing the notebook, there are some initial steps required for setup.

In [2]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade sagemaker --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sparkmagic 0.20.4 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.


To train and host on Amazon Sagemaker, we need to setup and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook instance as the AWS account role with SageMaker access. It has necessary permissions, including access to your data in S3. 

In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. Select Text Generation Model GTP-J 6B


In [4]:
model_id, model_version = "huggingface-textgeneration1-gpt-j-6b", "*"

## 3. Run Inference on the Pre-trained Model without finetuning

Using SageMaker, we can directly perform inference on the pre-trained [GPT-J 6B](https://huggingface.co/EleutherAI/gpt-j-6b) model. GPT-J 6B  is an open source 6 billion parameter model released by Eleuther AI. GPT-J 6B has been trained on a large corpus of text data ([the Pile](https://pile.eleuther.ai/) dataset) and is capable of performing various natural language processing tasks such as text generation, text classification, and text summarization. 

### 3.1. Retrieve Artifacts & Deploy an Endpoint

We retrieve the deploy_image_uri, deploy_source_uri, and base_model_uri for the pre-trained model. To host the pre-trained model, we create an instance of [`sagemaker.model.Model`](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) and deploy it.

In [5]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")

inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# Create the SageMaker model instance. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
model = Model(
    image_uri=deploy_image_uri,
    model_data=model_uri,
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)

# deploy the Model. TODO
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

---------------!

### 3.2. Query endpoint and parse response
The model takes a text string as input and predicts next words in the sequence. We use three of following input examples.

1. `This Form 10-K report shows that`
2. `We serve consumers through`
3. `Our vision is`

**The input examples are related to company's perforamnce in financial report. You will see the outputs from the model without finetuning are limited in providing insightful contents.**

In [6]:
import json
import boto3


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


def parse_response_multiple_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

In [7]:
parameters = {
    "max_length": 400,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 1,
}

res_gpt_before_finetune = []
for quota_text in [
    "This Form 10-K report shows that",
    "We serve consumers through",
    "Our vision is",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_before_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

This Form 10-K report shows that: (a) we, a Nevada corporation and Nevada corporation, have been registered with the Nevada Corporations Commission as an “small business” as defined by NRS 179.4160,179.3090,179.30,179.30 and 179.31 NRS;
(b) the number of our shareholders is limited to no more than 100 shareholders;
(c) the number of shares we issue is limited to no more than 500 shares;
(d) the principal business of our parent company is Nevada;
(e) the principal place of business of our company is Las Vegas, Nevada;
(f) we conduct our principal business in the State of Nevada;
(g) our place of incorporation is located in the State of Nevada;
(h) the Nevada corporation code is listed as N.C.A.C.O.1;
(i) our address is: _______________, Nevada;
(j) our place of incorporation is: ______________, Nevada;
7
(k) our: principal executive office is: _________, State of Nevada;
(l) our annual receipts, if applicable, are as follows:
(m) we are subject to federal income tax on our income;
(n) o

### 3.3. Clean up the endpoint

In [ ]:
# Delete the SageMaker endpoint and the attached resources
#base_model_predictor.delete_model()
#base_model_predictor.delete_endpoint()

## 4. Fine-tune the pre-trained model on a custom dataset

Fine-tuning refers to the process of taking a pre-trained language model and retraining it for a different but related task using specific data. This approach is also known as transfer learning, which involves transferring the knowledge learned from one task to another. Large language models (LLMs) like GPT-J 6B are trained on massive amounts of unlabeled data and can be fine-tuned on domain domain datasets, making the model perform better on that specific domain. 

We will use financial text from SEC filings to fine tune a LLM GPT-J 6B for financial applications. 



- **Input**: A train and an optional validation directory. Each directory contains a CSV/JSON/TXT file.
    - For CSV/JSON files, the train or validation data is used from the column called 'text' or the first column if no column called 'text' is found.
    - The number of files under train and validation (if provided) should equal to one.
- **Output**: A trained model that can be deployed for inference.
Below is an example of a TXT file for fine-tuning the Text Generation model. The TXT file is SEC filings of Amazon from year 2021 to 2022.

---
```
This report includes estimates, projections, statements relating to our
business plans, objectives, and expected operating results that are “forward-
looking statements” within the meaning of the Private Securities Litigation
Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E
of the Securities Exchange Act of 1934. Forward-looking statements may appear
throughout this report, including the following sections: “Business” (Part I,
Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K),
and “Management’s Discussion and Analysis of Financial Condition and Results
of Operations” (Part II, Item 7 of this Form 10-K). These forward-looking
statements generally are identified by the words “believe,” “project,”
“expect,” “anticipate,” “estimate,” “intend,” “strategy,” “future,”
“opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will
continue,” “will likely result,” and similar expressions. Forward-looking
statements are based on current expectations and assumptions that are subject
to risks and uncertainties that may cause actual results to differ materially.
We describe risks and uncertainties that could cause actual results and events
to differ materially in “Risk Factors,” “Management’s Discussion and Analysis
of Financial Condition and Results of Operations,” and “Quantitative and
Qualitative Disclosures about Market Risk” (Part II, Item 7A of this Form
10-K). Readers are cautioned not to place undue reliance on forward-looking
statements, which speak only as of the date they are made. We undertake no
obligation to update or revise publicly any forward-looking statements,
whether because of new information, future events, or otherwise.

GENERAL

Embracing Our Future ...
```
---
SEC filings data of Amazon is downloaded from publicly available [EDGAR](https://www.sec.gov/edgar/searchedgar/companysearch). Instruction of accessing the data is shown [here](https://www.sec.gov/os/accessing-edgar-data).

### 4.1. Retrieve Training artifacts
Here, for the selected model, we retrieve the training docker container, the training algorithm source, the pre-trained model, and a python dictionary of the training hyper-parameters that the algorithm accepts with their default values. Note that the model_version="*" fetches the latest model. Also, we do need to specify the training_instance_type to fetch train_image_uri.

In [8]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

training_instance_type = "ml.g5.12xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

### 4.2. Set Training parameters
Now that we are done with all the setup that is needed, we are ready to fine-tune our Text Classification model. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator will launch the training job. 

There are two kinds of parameters that need to be set for training. 

The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training. We defined the training instance type above to fetch the correct train_image_uri. 
***
The second set of parameters are algorithm specific training hyper-parameters. It is also used for sepcifying the model name if we want to fine-tune on the model which is not present in the dropdown list.
***

In [9]:
# Sample training data is available in this bucket
data_bucket = f"jumpstart-cache-prod-{aws_region}"
data_prefix = "training-datasets/sec_data"

training_dataset_s3_path = f"s3://{data_bucket}/{data_prefix}/train/"
validation_dataset_s3_path = f"s3://{data_bucket}/{data_prefix}/validation/"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-tg-train"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

***
For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.
***

In [10]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epoch"] = "3"
hyperparameters["per_device_train_batch_size"] = "4"
print(hyperparameters)

{'epoch': '3', 'learning_rate': '6e-06', 'per_device_train_batch_size': '4', 'per_device_eval_batch_size': '8', 'warmup_ratio': '0.1', 'instruction_tuned': 'False', 'train_from_scratch': 'False', 'fp16': 'True', 'bf16': 'False', 'evaluation_strategy': 'steps', 'eval_steps': '20', 'gradient_accumulation_steps': '2', 'logging_steps': '10', 'weight_decay': '0.2', 'load_best_model_at_end': 'True', 'max_train_samples': '-1', 'max_val_samples': '-1', 'seed': '10', 'max_input_length': '-1', 'validation_split_ratio': '0.2', 'train_data_split_seed': '0', 'preprocessing_num_workers': 'None', 'max_steps': '-1', 'gradient_checkpointing': 'True', 'early_stopping_patience': '3', 'early_stopping_threshold': '0.0', 'adam_beta1': '0.9', 'adam_beta2': '0.999', 'adam_epsilon': '1e-08', 'max_grad_norm': '1.0', 'label_smoothing_factor': '0', 'logging_first_step': 'False', 'logging_nan_inf_filter': 'True', 'save_strategy': 'steps', 'save_steps': '500', 'save_total_limit': '1', 'dataloader_drop_last': 'False

### 4.3. Train with Automatic Model Tuning ([HPO](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)) <a id='AMT'></a>
***
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We will use a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) object to interact with Amazon SageMaker hyperparameter tuning APIs.
***

In [11]:
from sagemaker.tuner import ContinuousParameter

# Use AMT for tuning and selecting the best model
use_amt = False

# Define objective metric, based on which the best model will be selected.
amt_metric_definitions = {
    "metrics": [{"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"}],
    "type": "Minimize",
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.00001, 0.0001, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 2

### 4.4. Start Training
***
We start by creating the estimator object with all the required assets and then launch the training job.
***

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

metric_definitions = [
    {"Name": "train:loss", "Regex": "'loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:loss", "Regex": "'eval_loss': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:runtime", "Regex": "'eval_runtime': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:samples_per_second", "Regex": "'eval_samples_per_second': ([0-9]+\.[0-9]+)"},
    {"Name": "eval:eval_steps_per_second", "Regex": "'eval_steps_per_second': ([0-9]+\.[0-9]+)"},
]


# Create SageMaker Estimator instance
tg_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    metric_definitions=metric_definitions,
)

if use_amt:
    hp_tuner = HyperparameterTuner(
        tg_estimator,
        amt_metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        amt_metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=amt_metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"train": training_dataset_s3_path, "validation": validation_dataset_s3_path})
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    tg_estimator.fit(
        {"train": training_dataset_s3_path, "validation": validation_dataset_s3_path}, logs=True
    )

INFO:sagemaker:Creating training-job with name: jumpstart-example-huggingface-textgener-2023-07-14-14-32-05-455


2023-07-14 14:32:05 Starting - Starting the training job...
2023-07-14 14:32:32 Starting - Preparing the instances for training............
2023-07-14 14:34:12 Downloading - Downloading input data.......................................................46%|████▌     | 44/96 [23:35<28:34, 32.97s/it]
52%|█████▏    | 50/96 [26:38<23:24, 30.53s/it]
{'loss': 0.6544, 'learning_rate': 6e-06, 'epoch': 1.54}
83%|████████▎ | 80/96 [42:04<07:58, 29.90s/it]
{'loss': 0.3435, 'learning_rate': 6e-06, 'epoch': 2.46}
83%|████████▎ | 80/96 [42:04<07:58, 29.90s/it]
[INFO|trainer.py:3129] 2023-07-14 15:34:07,859 >> ***** Running Evaluation *****
[INFO|trainer.py:3129] 2023-07-14 15:34:07,859 >> ***** Running Evaluation *****
[INFO|trainer.py:3131] 2023-07-14 15:34:07,860 >>   Num examples = 198
[INFO|trainer.py:3131] 2023-07-14 15:34:07,860 >>   Num examples = 198
[INFO|trainer.py:3134] 2023-07-14 15:34:07,861 >>   Batch size = 8
[INFO|trainer.py:3134] 2023-07-14 15:34:07,861 >>   Batch size = 8
100%|██████

### 4.5. Extract Training performance metrics
***
Performance metrics such as training loss and validation accuracy/loss can be accessed through cloudwatch while the training. We can also fetch these metrics and analyze them within the notebook
***

In [14]:
from sagemaker import TrainingJobAnalytics

if use_amt:
    training_job_name = hp_tuner.best_training_job()
else:
    training_job_name = tg_estimator.latest_training_job.job_name

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df.head(10)

,timestamp,metric_name,value
0,0.0,train:loss,1.696900
1,300.0,train:loss,1.391000
2,660.0,train:loss,1.130000
3,960.0,train:loss,0.812800
4,1260.0,train:loss,0.654400
5,1560.0,train:loss,0.561400
6,1920.0,train:loss,0.457000
7,2220.0,train:loss,0.343500
8,2520.0,train:loss,0.344400
9,0.0,eval:loss,1.271484


## 4.6. Deploy & run Inference on the fine-tuned model
***
A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the class label of an input sentence. We follow the same steps as in [3. Run inference on the pre-trained model](#3.-Run-inference-on-the-pre-trained-model). We start by retrieving the artifacts for deploying an endpoint. However, instead of base_predictor, we  deploy the `tg_estimator` that we fine-tuned.
***

In [15]:
inference_instance_type = "ml.g5.12xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)

endpoint_name_after_finetune = name_from_base(f"jumpstart-example-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = (hp_tuner if use_amt else tg_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    image_uri=deploy_image_uri,
    endpoint_name=endpoint_name_after_finetune,
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py39.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.g5.12xlarge.
INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2023-07-15-06-00-19-688
INFO:sagemaker:Creating endpoint-config with name jumpstart-example-huggingface-textgener-2023-07-15-06-00-19-688
INFO:sagemaker:Creating endpoint with name jumpstart-example-huggingface-textgener-2023-07-15-06-00-19-688


-------------!

Next, we query the finetuned model using the same set of examples above, parse the response and print the predictions.

In [17]:
import json
import boto3


def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response


def parse_response_multiple_texts(query_response):
    generated_text = []
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]

The outputs from fine-tune model are generated as below. We can see that after being fine-tuned, the model can generate more insightful contents related to financial domain.

In [18]:
parameters = {
    "max_length": 400,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 1,
}

res_gpt_finetune = []
for quota_text in [
    "This Form 10-K report shows that",
    "We serve consumers through",
    "Our vision is",
]:
    payload = {"text_inputs": f"{quota_text}:", **parameters}

    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name_after_finetune
    )
    generated_texts = parse_response_multiple_texts(query_response)[0]["generated_text"]
    res_gpt_finetune.append(generated_texts)
    print(generated_texts)
    print("\n")

This Form 10-K report shows that:(1)    We are a public company, as defined by Rule 13a-15(c) of the SecuritiesExchange Act of 1934.  (2)    The following is a summary of certain material contract provisions thataffect our ability to meet our contractual commitments. This summary is notcomplete and is qualified in its entirety by reference to the entirecontract.  (1)    The agreements may be amended, modified, terminated, or otherwisechanged only by written instrument signed by the party or parties affectedand, in the case of the JV Agreements, by the parties thereto.  (2)    In the event we fail to meet our contractual obligations for anyreason, including as a result of acts of God, extreme weather, fire, flood,earthquakes, acts of war or terrorism, labor or material shortages, acts ofgovernment or regulatory authority, or other events, the other party has theright to terminate this Agreement by notifying us of such termination. Thereafter,the other party may elect to pursue any avail

We compare the outputs between the model before fine-tuning and after fine-tuning.

In [19]:
import pandas as pd

pd.DataFrame(
    {
        "Input example": [
            "This Form 10-K report shows that",
            "We serve consumers through",
            "Our vision is",
        ],
        "Output before finetuning": res_gpt_before_finetune,
        "Output after finetuning": res_gpt_finetune,
    }
)

,Input example,Output before finetuning,Output after finetuning
0,This Form 10-K report shows that,"This Form 10-K report shows that: (a) we, a Ne...",This Form 10-K report shows that:(1) We are...
1,We serve consumers through,We serve consumers through:\n\n\n\n\nWe serve ...,We serve consumers through: Amazon Marketplace...
2,Our vision is,Our vision is: “We believe that all humans are...,Our vision is:To be Earth’s most customer-cent...


In [20]:
print(res_gpt_before_finetune[0])
print(res_gpt_finetune[1])

This Form 10-K report shows that: (a) we, a Nevada corporation and Nevada corporation, have been registered with the Nevada Corporations Commission as an “small business” as defined by NRS 179.4160,179.3090,179.30,179.30 and 179.31 NRS;
(b) the number of our shareholders is limited to no more than 100 shareholders;
(c) the number of shares we issue is limited to no more than 500 shares;
(d) the principal business of our parent company is Nevada;
(e) the principal place of business of our company is Las Vegas, Nevada;
(f) we conduct our principal business in the State of Nevada;
(g) our place of incorporation is located in the State of Nevada;
(h) the Nevada corporation code is listed as N.C.A.C.O.1;
(i) our address is: _______________, Nevada;
(j) our place of incorporation is: ______________, Nevada;
7
(k) our: principal executive office is: _________, State of Nevada;
(l) our annual receipts, if applicable, are as follows:
(m) we are subject to federal income tax on our income;
(n) o

---
Next, we clean up the deployed endpoint.

---

In [ ]:
# Delete the SageMaker endpoint and the attached resources
#finetuned_predictor.delete_model()
#finetuned_predictor.delete_endpoint()

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/introduction_to_amazon_algorithms|jumpstart-foundation-models|domain-adaption-finetuning-gpt-j-6b.ipynb)
